In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from PPO import PPO, PPOContinuous
import pdb

env = gym.make("Pendulum-v0")

env.seed(10)

state_size = env.observation_space.shape[0]
action_size =env.action_space.shape[0]

print(state_size)
print(action_size)

# PPO Settings
update_every = 2000
num_learn = 40
win_condition = 0

# Agent settings
hidden_size=128
epsilon=0.2
entropy_beta=0.01
gamma=0.99
lr=0.0003

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

agent = PPOContinuous(state_size, action_size, hidden_size=hidden_size, epsilon=epsilon, entropy_beta=entropy_beta, gamma=gamma, lr=lr)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


3
1


In [ ]:
def train(n_episodes=4000, max_t=1500):
  steps = 0
  scores_deque = deque(maxlen=100)
  scores = []
  average_scores = []
  max_score = -np.Inf

#   agent = PPO(state_size, action_size, hidden_size=hidden_size, epsilon=epsilon, entropy_beta=entropy_beta, gamma=gamma, lr=lr)

  for episode in range(1, n_episodes+1):
    state = env.reset()
    score = 0
    
    for t in range(max_t):
      steps += 1

      actions_tensor, log_prob = agent.act(torch.FloatTensor(state).to(device))
      actions = actions_tensor.cpu().data.numpy().flatten()
      next_state, reward, done, _ = env.step(actions_tensor.cpu().numpy())

      agent.mem.add(torch.FloatTensor(state), actions, reward, log_prob, done)

      # Update 
      state = next_state
      score += reward.item()

      if steps >= update_every:
        agent.learn(num_learn)
        agent.mem.clear()
        steps = 0

      if done:
        break
    
    # Book Keeping
    scores_deque.append(score)
    scores.append(score)
    average_scores.append(np.mean(scores_deque))
      

    print("\rEpisode {}	Average Score: {:.2f}	Score: {:.2f}".format(episode, np.mean(scores_deque), score), end="")

    if episode % 100 == 0:
      print("\rEpisode {}	Average Score: {:.2f}".format(episode, np.mean(scores_deque)))   
    
    if np.mean(scores_deque) > win_condition:
      print("\rEnvironment Solved in {} episodes!".format(episode))
      break


  return scores, average_scores

scores, average_scores = train()

Episode 9	Average Score: -1353.71	Score: -1258.84

/home/ubuntu/rl_lib/Policy Gradient/src/PPO/PPO.py:125: UserWarning: Using a target size (torch.Size([2000])) that is different to the input size (torch.Size([2000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = -torch.min(surrogate_1, surrogate_2) + 0.5*F.mse_loss(values, discounted_returns) - self.entropy_beta*entropy


Episode 100	Average Score: -1304.35	Score: -1558.95
Episode 200	Average Score: -1238.16	Score: -1023.26
Episode 300	Average Score: -1098.61	Score: -881.933
Episode 400	Average Score: -1005.74	Score: -975.379
Episode 500	Average Score: -1102.78	Score: -1043.61
Episode 600	Average Score: -1122.40	Score: -1362.99
Episode 621	Average Score: -1149.33	Score: -1198.21

In [ ]:
torch.save(agent.model.state_dict(), "pendulum_ppo_model.pth")
torch.save(agent.model_old.state_dict(), "pendulum_ppo_model_old.pth")

In [ ]:
plt.plot(scores)
plt.plot(average_scores)

In [ ]:
for episode in range(3):
    state = env.reset()
    score = 0
    input()

    for t in range(1500):
        actions_tensor, log_prob = agent.act(torch.FloatTensor(state))
        actions = actions_tensor.cpu().data.numpy().flatten()
        next_state, reward, done, _ = env.step(actions_tensor)

        
        score += reward
        
        env.render()
        
#         if done:
#             print(score)
#             break;
        
        state = next_state

env.close()